In [ ]:
%%bash
echo $CMSSW_BASE
echo $PWD

In [ ]:
%%bash
uname -a
whoami

edmFileUtil root://eoscms.cern.ch///store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/593/00000/C0D83139-4A6E-E711-B1A9-02163E0137FC.root
edmFileUtil root://eoscms.cern.ch///store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/593/00000/4269D74E-4A6E-E711-847B-02163E01A487.root

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import time

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from DataFormats.FWLite import Handle, Events

In [ ]:
#LS50
events2016HL = "root://cms-xrd-global.cern.ch///store/data/Run2016H/JetHT/AOD/PromptReco-v2/000/283/408/00000/AE49FDAB-8398-E611-9B98-02163E014388.root"
#LS1000
events2016LL = "root://cms-xrd-global.cern.ch///store/data/Run2016H/JetHT/AOD/PromptReco-v2/000/283/408/00000/88FB6BA5-7B98-E611-B25E-FA163EE1CC08.root"

events2017B_HL = "root://cms-xrd-global.cern.ch///store/data/Run2017B/SingleMuon/AOD/PromptReco-v2/000/299/061/00000/5C4984B0-B86A-E711-837F-02163E01A792.root"
events2017B_LL = "root://cms-xrd-global.cern.ch///store/data/Run2017B/SingleMuon/AOD/PromptReco-v2/000/299/067/00000/B4146406-F66A-E711-B9C9-02163E014389.root"

label = "generalTracks"
quality = "highPurity"

In [ ]:
def nt(fevents) :
    events = Events(fevents)
    tracks = Handle("std::vector<reco::Track>")
    nt = []
    for i in range(0,1000) : # events.size()):
      a= events.to(i)
      if (i%500==0) : print "Event", i 
      a=events.getByLabel(label, tracks)
      nt.append(tracks.product().size())
    print len(nt)
    return nt

In [ ]:
nt = nt(events2017B_HL)
fig, ax = plt.subplots()
ax.hist(nt,histtype='step', fill=False)
ax.set_title('Number of tracks in event')
ax.set_xlabel('#tracks')
ax.set_ylabel('#events')
plt.yscale('log', nonposy='clip')
plt.show()

In [ ]:
y,x = np.histogram(nt,np.linspace(0.,3000.,300))
x.resize(len(y))
plt.step(x,y,where='post',label='N tracks')
plt.grid(True)
plt.legend(loc='upper right')
plt.show()

In [ ]:
# testBit() returns a nonzero result, 2**offset, if the bit at 'offset' is one.
def testBit(int_type, offset):
    mask = 1 << offset
    return(int_type & mask)

def onlyBit(int_type, offset):
    mask = 1 << offset
    return(int_type ^ mask)

In [ ]:
def loadTk(fevents) :
    print fevents
    events = Events(fevents)
    print events.size()
    tracks = Handle("std::vector<reco::Track>")
    tkParsA = []
    tkHitsA = []
    for i in range(0,events.size()) :
      if (len(tkParsA) > 1000000) : break
      a= events.to(i)
      if (i%500==0) : 
        id = events.object().id()
        evid = '{:d}:{:d}:{:d}'.format(int(id.run()),int(id.luminosityBlock()), int(id.event()))
        print "Event", i , evid, len(tkParsA)
      a=events.getByLabel(label, tracks)
      for tk in tracks.product() :
        if (not tk.quality(tk.qualityByName(quality))) : continue
        pattern = tk.hitPattern()
        tkParsA.append([tk.eta(),tk.phi(),tk.pt(), tk.ndof(),tk.chi2()])
        tkHitsA.append([pattern.numberOfValidHits(),pattern.numberOfValidPixelHits(),tk.originalAlgo()-4,tk.algoMaskUL()])

    print len(tkParsA)
    tkPars = np.array(zip(*tkParsA), dtype=np.float)
    print len(tkPars)
    tkHits = np.array(zip(*tkHitsA), dtype=np.int)
    print len(tkHits)
    return (tkPars,tkHits)

In [ ]:
def loadFile() :
    loaded = np.load('events2017B_LL.npz')
    return loaded['tkPars'], loaded['tkHits']

In [ ]:
fromAOD = False
tkPars, tkHits = loadTk(events2017B_LL) if fromAOD else loadFile()

In [ ]:
if fromAOD :
    np.savez_compressed('events2017B_LL',tkPars=tkPars, tkHits=tkHits)

In [ ]:
%%bash
ls -l events2017B_LL*

In [ ]:
def nhit(var,tkPars, tkHits) :
    hp = np.greater(tkPars[2],4.)
    notQ = np.equal(tkHits[2],18)|np.equal(tkHits[2],1)|np.equal(tkHits[2],2)|np.equal(tkHits[2],3)
    quad = np.equal(tkHits[2],0)|np.equal(tkHits[2],19)|np.equal(tkHits[2],20)
    mu = np.not_equal(testBit(tkHits[3],4+9),0)
    nbins=62
    yn,x = np.histogram(tkPars[var],np.linspace(-3.1,3.1,nbins),weights=1.*tkHits[0]*hp)
    ya,x = np.histogram(tkPars[var],np.linspace(-3.1,3.1,nbins),weights=1.*hp)
    x.resize(len(ya))
    return x,yn/ya,ya
    

In [ ]:
print tkPars[4][0],tkHits[0][0],tkHits[1][0],
eta=0
phi=1
pt=2

var = eta
x,y,ya = nhit(eta,tkPars, tkHits)

plt.step(x,y,where='post',label='run xyz, lumi??')
plt.grid(True)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlim(-3.1, 3.1)
plt.xlabel('eta')
plt.ylabel('<Nhits>')
plt.show()